In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sln
import matplotlib.pyplot as plt
%matplotlib inline
import gc
from scipy.sparse import hstack

In [ ]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

In [ ]:
#check data is balanced
train['Complaint-Status'].value_counts()

In [ ]:
train.head()

In [ ]:
#missing values
train.isnull().sum()

In [ ]:
# #nlp+machine learnings
# plt.figure(figsize=(10,10))
# sln.countplot(train['Complaint-Status'],hue=train['Consumer-disputes'])
# # plt.tight_layout()

In [ ]:
# plt.figure(figsize=(10,10))
# sln.countplot(train['Complaint-Status'],hue=train['Transaction-Type'])
# plt.tight_layout()

In [ ]:
train['Company-response']=train['Company-response'].fillna('Nan')

In [ ]:
#rain['merged']=train['Transaction-Type']+'_'+train['Complaint-reason']+'_'+train['Company-response']+"_"+train['Consumer-complaint-summary']

In [ ]:
#lets not merge the text data just clean it

In [ ]:
 from nltk.tokenize import word_tokenize
 from nltk.corpus import stopwords
 import re
import nltk
punctuation=re.compile(r'[/*x*.?!,":;()|0-9]')


In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words(['english','spanish','french'])
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
stemmer_spanish = SnowballStemmer('spanish')
stemmer_english = SnowballStemmer('english')
stemmer_french = SnowballStemmer('french')

BASIC preprocessing

In [ ]:
def converter(df):
    #remove tags
    
    
    # convert to lower
    df['Consumer-complaint-summary'] =df['Consumer-complaint-summary'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    # remove punctuations
    
    df['Consumer-complaint-summary'] = df['Consumer-complaint-summary'].apply(lambda x: punctuation.sub('',x))
    
    
    # remove stop words
    
    df['Consumer-complaint-summary'] = df['Consumer-complaint-summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    
    
    # stemming
    
    #df['merged']= df['merged'].apply(lambda x:" ".join(stemmer_spanish.stem(token) for token in x.split(" ")))
    df['Consumer-complaint-summary']= df['Consumer-complaint-summary'].apply(lambda x:" ".join(stemmer_english.stem(token) for token in x.split(" ")))
    #df['merged']= df['merged'].apply(lambda x:" ".join(stemmer_french.stem(token) for token in x.split(" ")))
    
    
  

    
    return df
    

In [ ]:
%%time
train_new=converter(train)
test_new=converter(test)

In [ ]:
train_new['Consumer-complaint-summary'][0]

**BASE MODEL**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

# nb = Pipeline([
#                ('tfidf', TfidfVectorizer(max_features=2000,analyzer='word',ngram_range=(2,3))),
               
#               ])
# X_tfid=nb.fit_transform(train_new['merged'])
# X_tfdf=pd.DataFrame(X_tfid.toarray())

In [ ]:
train_new['Date-sent-to-company']=pd.to_datetime(train_new['Date-sent-to-company'])
train_new['Date-received']=pd.to_datetime(train_new['Date-received'])


In [ ]:
dummy=pd.get_dummies(train_new['Consumer-disputes'],drop_first=True)
train_new['date_diff']=train_new['Date-sent-to-company']-train_new['Date-received']
train_new['date_diff']=train_new['date_diff'].astype('int')/365
train_Comb=pd.concat([X_tfdf,dummy,train_new['date_diff']],axis=1)

In [ ]:
X=train_Comb
y=train_new['Complaint-Status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn import metrics
# clf=RandomForestClassifier(n_estimators=1000,class_weight='balanced')
# scores = cross_val_score(clf, X, y, cv=5, scoring='f1_weighted')
# print(scores)
from sklearn.metrics import classification_report
clf.fit(X_train,y_train)

#nt('accuracy %s' % accuracy_score(y_pred, y_test))clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))
# print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred))


In [ ]:
test['Company-response']=test['Company-response'].fillna('Nan')
test['merged']=test['Transaction-Type']+test['Complaint-reason']+test['Company-response']+test['Consumer-complaint-summary']

In [ ]:
test.isnull().sum()

In [ ]:
test_new=converter(test)

In [ ]:
test_tfid=nb.fit_transform(test_new['merged'])
test_tfdf=pd.DataFrame(test_tfid.toarray())

In [ ]:
test_new['Date-sent-to-company']=pd.to_datetime(test_new['Date-sent-to-company'])
test_new['Date-received']=pd.to_datetime(test_new['Date-received'])


In [ ]:
dummy_test=pd.get_dummies(test_new['Consumer-disputes'],drop_first=True)
test_new['date_diff']=test_new['Date-sent-to-company']-test_new['Date-received']
test_new['date_diff']=test_new['date_diff'].astype('int')/365
test_Comb=pd.concat([test_tfdf,dummy_test,test_new['date_diff']],axis=1)

In [ ]:
results=clf.predict(test_Comb)

In [ ]:
my_submission=pd.DataFrame({'Complaint-ID':test['Complaint-ID'],'Complaint-Status':results})
my_submission.to_csv('sub1.csv',index=False)

In [ ]:
# from sklearn.linear_model import SGDClassifier

# sgd = Pipeline([('vect', CountVectorizer()),
#                 ('tfidf', TfidfTransformer()),
#                 ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
#                ])
# sgd.fit(X_train, y_train)



# y_pred = sgd.predict(X_test)

# print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred))

In [ ]:
# length=train_new['merged'].apply(lambda x:len(x))
# print("max length : {}".format(length.max()))
# print("min length : {}".format(length.min()))


In [ ]:
epochs=4
max_review_length1=100
max_review_length2=500
max_review_length3=2000
max_review_length4=10000
n_unique=5000

pad_type = trunc_type = 'pre'

# n_conv = 300 # filters, a.k.a. kernels
# k_conv = 4 # kernel length
# drop_embed = 0.2 

# # LSTM layer architecture:
# n_lstm = 256 
# drop_lstm = 0.2

In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.layers.wrappers import Bidirectional # new! 
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

In [ ]:
%%time
max_features = 5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_new['Consumer-complaint-summary'])
list_tokenized_train=tokenizer.texts_to_sequences(train_new['Consumer-complaint-summary'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
gle = LabelEncoder()
genre_labels = gle.fit_transform(train_new['Complaint-Status'])
genre_mappings = {index: label for index, label in 
                  enumerate(gle.classes_)}
train_new['GenreLabel'] = genre_labels

In [ ]:
from sklearn.model_selection import train_test_split
X=list_tokenized_train
y=keras.utils.np_utils.to_categorical(train['GenreLabel'])
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)


In [ ]:
x_train = pad_sequences(X_train, maxlen=max_review_length2, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(X_test, maxlen=max_review_length2, padding=pad_type, truncating=trunc_type, value=0)

In [ ]:
# model = Sequential()
# model.add(Embedding(5000, 128, input_length=max_review_length2)) 
# model.add(SpatialDropout1D(0.7))
# model.add(Bidirectional(LSTM(500, dropout=drop_lstm)))

# model.add(Dense(5, activation='softmax'))

In [ ]:
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# output_dir = 'model_output/biLSTM'
# modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

In [ ]:
# model.fit(x_train, y_train, epochs=10, batch_size=500,validation_data=(x_valid,y_test),callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

# Trying Light gbm with word vectorizers and character vectorizers

In [ ]:
train_text = train_new['Consumer-complaint-summary']
test_text = test_new['Consumer-complaint-summary']


In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
   
    ngram_range=(1, 1),
    max_features=5000)
word_vectorizer.fit(train_text)

In [ ]:
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)


In [ ]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(2, 4),
    max_features=5000)
char_vectorizer.fit(train_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)


In [ ]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [ ]:
#preparing inputs 
X_tfdf=pd.DataFrame(train_features.toarray())
train_new['Date-sent-to-company']=pd.to_datetime(train_new['Date-sent-to-company'])
train_new['Date-received']=pd.to_datetime(train_new['Date-received'])
dummy=pd.get_dummies(train_new[['Consumer-disputes','Transaction-Type','Complaint-reason','Company-response']],drop_first=True)
train_new['date_diff']=train_new['Date-sent-to-company']-train_new['Date-received']
train_new['date_diff']=train_new['date_diff'].astype('int')/365
train_Comb=pd.concat([X_tfdf,dummy,train_new['date_diff']],axis=1)





In [ ]:
# from sklearn.preprocessing import LabelEncoder
# gle = LabelEncoder()
# genre_labels = gle.fit_transform(train_new['Complaint-Status'])
# genre_mappings = {index: label for index, label in 
#                   enumerate(gle.classes_)}
# train_new['GenreLabel'] = genre_labels
y=train_new['Complaint-Status']


X_train, X_test,y_train, y_test = train_test_split(train_features,y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
clf=RandomForestClassifier(n_estimators=500,class_weight='balanced')
scores = cross_val_score(clf, train_features, y, cv=5, scoring='f1_weighted')
print(scores)

##using previously trained embedding

In [ ]:
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, 300, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])